In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import tensorflow as tf
from keras.models import Model
import tensorflow
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.applications import MobileNetV2,DenseNet121,EfficientNetV2B0
from tensorflow.keras.applications import NASNetMobile
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Lambda,Concatenate
import tensorflow.keras.backend as K
from tensorflow.keras import Input
from tensorflow.keras.utils import Sequence
from pathlib import Path
import numpy as np

import tensorflow as tf
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense, Dropout, Input, Lambda, Flatten, Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing import image
from keras import backend as K
import os # system-wide functions
#os.listdir('/kaggle/input/BloodPressureDataset')
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense,Input,BatchNormalization
from tensorflow.keras.layers import Dropout,Concatenate, Conv2D,Flatten, MaxPooling2D
from tensorflow.keras.models import Sequential,Model

from tensorflow.keras import optimizers
import sys
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, RMSprop, Adadelta
from tensorflow.keras.layers import BatchNormalization
from keras.regularizers import l2
import random
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2

In [3]:
train_path_csv = '/content/drive/MyDrive/age_gender/train.csv'
test_path_csv = '/content/drive/MyDrive/age_gender/test.csv'

In [4]:
train_df = pd.read_csv(train_path_csv)
test_df =pd.read_csv(test_path_csv)

In [5]:
print(train_df.shape,test_df.shape)

(538, 3) (60, 3)


In [6]:
test_df.head(5)

,image_name,gender,age
0,34_13_M.jpg,M,13
1,187_19_M.jpg,M,19
2,20_24_M.jpg,M,24
3,231_22_M.jpg,M,22
4,234_22_M.jpg,M,22


In [7]:
train_path = "/content/drive/MyDrive/age_gender/images"
test_path = "/content/drive/MyDrive/age_gender/images"

In [8]:
i_shape = 224

In [9]:
BATCH_SIZE = 16

In [10]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import math

# Here, `x_set` is list of path to the images

In [11]:

# and `y_set` are the associated classes.

class TrainDataSequence(tf.keras.utils.Sequence):

    def __init__(self, image_set, batch_size):
        self.image_set = image_set
        self.batch_size = batch_size
        self.train_path = train_path
        self.output_size = (224, 224)
        self.i_shape = self.output_size[0]

    def __len__(self):
        return math.ceil(len(self.image_set) / self.batch_size)

    def __getitem__(self, idx):
        batch_image_set = self.image_set[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        input_set = np.zeros((self.batch_size, *self.output_size, 3))
        # (x, y, h, w)
        gender_set = np.zeros((self.batch_size, 1))
        age_set = np.empty((self.batch_size,1))
        gender_dict = {'F':0, 'M' : 1}
        image = []
        gender = []
        age = []
        for i,file_name in enumerate(batch_image_set):
          img = cv2.imread(os.path.join(self.train_path,file_name))
          img = cv2.resize(img,(self.i_shape,self.i_shape))
          input_set[i,] = img
          gender_set[i][0] = gender_dict[file_name.split('.')[0].split('_')[-1]]
          age_set[i][0] = int(file_name.split('.')[0].split('_')[1])
        return input_set, {"gender_output" : gender_set, "age_output":age_set}

In [12]:
train_data_sequence =  TrainDataSequence(train_df['image_name'], batch_size = BATCH_SIZE)

In [13]:
class TestDataSequence(tf.keras.utils.Sequence):

    def __init__(self, image_set, batch_size):
        self.image_set = image_set
        self.batch_size = batch_size
        self.train_path = train_path
        self.output_size = (224, 224)
        self.i_shape = self.output_size[0]

    def __len__(self):
        return math.ceil(len(self.image_set) / self.batch_size)

    def __getitem__(self, idx):
        batch_image_set = self.image_set[idx * self.batch_size:(idx + 1) *
        self.batch_size]
        input_set = np.zeros((self.batch_size, *self.output_size, 3))
        # (x, y, h, w)
        gender_set = np.zeros((self.batch_size, 1))
        age_set = np.zeros((self.batch_size,1))
        gender_dict = {'F':0, 'M' : 1}
        image = []
        gender = []
        age = []
        for i,file_name in enumerate(batch_image_set):
          img = cv2.imread(os.path.join(self.train_path,file_name))
          img = cv2.resize(img,(self.i_shape,self.i_shape))
          input_set[i,] = img
          gender_set[i][0] = gender_dict[file_name.split('.')[0].split('_')[-1]]
          age_set[i][0] = int(file_name.split('.')[0].split('_')[1])
        return input_set, {"gender_output" : gender_set, "age_output":age_set}

In [14]:
test_data_sequence =  TestDataSequence(test_df['image_name'], batch_size = BATCH_SIZE)

In [15]:
class DataGenerator(Sequence):

  def __init__(self, csv_file, base_dir, output_size, shuffle=False, batch_size=10):
    """
    Initializes a data generator object
      :param csv_file: file in which image names and numeric labels are stored
      :param base_dir: the directory in which all images are stored
      :param output_size: image output size after preprocessing
      :param shuffle: shuffle the data after each epoch
      :param batch_size: The size of each batch returned by __getitem__
    """
    self.df = pd.read_csv(csv_file)
    self.base_dir = base_dir
    self.output_size = output_size
    self.shuffle = shuffle
    self.batch_size = batch_size
    self.on_epoch_end()

  def on_epoch_end(self):
    self.indices = np.arange(len(self.df))
    if self.shuffle:
      np.random.shuffle(self.indices)

  def __len__(self):
    return int(len(self.df) / self.batch_size)

  def __getitem__(self, idx):
    ## Initializing Batch
    #  that one in the shape is just for a one channel images
    # if you want to use colored images you might want to set that to 3
    X = np.empty((self.batch_size, *self.output_size, 3))
    # (x, y, h, w)
    y = np.empty((self.batch_size, 4, 1))
    print(y.shape)
    # get the indices of the requested batch
    indices = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

    for i, data_index in enumerate(indices):
      img_path = os.path.join(self.base_dir,
                  self.df.iloc[data_index, 0])

      #img = mpimg.imread()
      #img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # to reduce it to one channel to match the shape
      ## this is where you preprocess the image
      ## make sure to resize it to be self.output_size

      label = self.df.iloc[data_index, 1:].to_numpy()
      ## if you have any preprocessing for
      ## the labels too do it here

      #X[i,] = img
      #y[i] = label

    return X, y



In [16]:
#train_gen = DataGenerator("data.csv", "data", (244, 244), batch_size=20, shuffle=True)

In [17]:
def extractor_model(input_shape):

    #vgg_16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
#     vgg_16 = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape)
    #vgg_16.trainable = True
    #x = vgg_16.output
    mob = ResNet50V2(weights = 'imagenet')
    #mob = Xception(weights = 'imagenet')
    #mob = MobileNetV2(weights = 'imagenet')
    #mob = DenseNet121(weights = 'imagenet')
    #mob = NASNetMobile(weights = 'imagenet')
    #mob = EfficientNetV2B0(weights = 'imagenet')
    #mob = InceptionV3(weights = 'imagenet')
    mob.trainable = True
    x = mob.output
    x = Flatten(name="flatten")(x)
    # x = Dense(512, activation='relu')(x)
    # x = Dropout(0.3)(x)
    #output_layer = Dense(128, activation='relu')(x)
#     x = Dense(64, activation='relu')(x)

    # output_layer = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=mob.input, outputs=x)

    # model.compile(optimizer=optimizer,
    #               loss='binary_crossentropy',
    #               metrics=['accuracy'])

    return model

In [18]:
def create_model(input_shape):
  #print(input_shape)
  base_model = extractor_model(input_shape)
  #print(base_model.summary())
  input_image = Input(shape=input_shape)
  feat_image = base_model(input_image)


  gender = Dense(800,activation="relu")(feat_image)
  gender = Dropout(0.5)(gender)
  '''
  gender = Dense(600,activation="relu")(gender)
  gender = Dropout(0.3)(gender)
  gender = Dense(400,activation="relu")(gender)
  gender = Dropout(0.6)(gender)
  '''
  gender = Dense(200,activation="relu")(gender)
  gender = Dropout(0.5)(gender)
  gender = Dense(100,activation="relu")(gender)
  gender = Dropout(0.2)(gender)
  '''
  gender = Dense(50,activation="relu")(gender)
  gender = Dropout(0.5)(gender)
  gender = Dense(25,activation="relu")(gender)
  gender = Dropout(0.7)(gender)
  '''
  gender = Dense(12,activation="relu")(gender)
  gender = Dropout(0.5)(gender)
  gender = Dense(1,activation="sigmoid",name = "Gender_output_layer")(gender)


  age = Dense(800,activation="relu")(feat_image)
  age = Dropout(0.5)(age)
  '''
  age = Dense(600,activation="relu")(age)
  age = Dropout(0.3)(age)
  age = Dense(400,activation="relu")(age)
  age = Dropout(0.6)(age)
  '''
  age = Dense(200,activation="relu")(age)
  age = Dropout(0.5)(age)
  age = Dense(100,activation="relu")(age)
  age = Dropout(0.2)(age)
  '''
  age = Dense(50,activation="relu")(age)
  age = Dropout(0.5)(age)
  age = Dense(25,activation="relu")(age)
  age = Dropout(0.7)(age)
  '''
  age = Dense(12,activation="relu")(age)
  age = Dropout(0.5)(age)
  age = Dense(1,activation="linear",name = 'Age_output_layer')(age)




  model = Model(input_image, outputs= {"gender_output" : gender, "age_output": age})
  return model

In [19]:
def custom_create_model_given(input_shape):
  input_image = Input(shape=input_shape)
  '''
  #mix age-gender section
  mix_sec1 = Conv2D(filters=512, kernel_size=3, activation='relu')(input_image)
  mix_sec1 = Conv2D(filters=512, kernel_size=3, activation='relu')(mix_sec1)
  mix_sec1 = BatchNormalization()(mix_sec1)

  mix_sec2 = MaxPooling2D(pool_size = 2)(mix_sec1)

  mix_sec3 = Conv2D(filters=256, kernel_size=3, activation='relu')(mix_sec2)
  mix_sec3 = Conv2D(filters=256, kernel_size=3, activation='relu')(mix_sec3)
  mix_sec3 = BatchNormalization()(mix_sec3)

  mix_sec4 = MaxPooling2D(pool_size = 2)(mix_sec3)

  mix_sec5 = Conv2D(filters=128, kernel_size=3, activation='relu')(mix_sec4)
  mix_sec5 = Conv2D(filters=128, kernel_size=3, activation='relu')(mix_sec5)
  mix_sec5 = BatchNormalization()(mix_sec5)

  mix_sec6 = MaxPooling2D(pool_size = 2)(mix_sec5)

  mix_sec7 = Conv2D(filters=64, kernel_size=3, activation='relu')(mix_sec6)
  mix_sec7 = Conv2D(filters=64, kernel_size=3, activation='relu')(mix_sec7)
  mix_sec7 = BatchNormalization()(mix_sec7)

  mix_sec8 = MaxPooling2D(pool_size=2)(mix_sec7)
  '''


  #only gender section
  gender_sec1 = Conv2D(filters=512, kernel_size=3, activation='relu')(input_image)
  #gender_sec1 = Conv2D(filters=512, kernel_size=3, activation='relu')(gender_sec1)
  gender_sec1 = BatchNormalization()(gender_sec1)

  gender_sec2 = MaxPooling2D(pool_size = 2)(gender_sec1)
  #gender_sec2 = Concatenate(axis= 1)([gender_sec2,mix_sec2])

  gender_sec3 = Conv2D(filters=256, kernel_size=3, activation='relu')(gender_sec2)
  #gender_sec3 = Conv2D(filters=256, kernel_size=3, activation='relu')(gender_sec3)
  gender_sec3 = BatchNormalization()(gender_sec3)

  gender_sec4 = MaxPooling2D(pool_size = 2)(gender_sec3)
  #gender_sec4 = Concatenate(axis= 1)([gender_sec4,mix_sec4])

  gender_sec5 = Conv2D(filters=128, kernel_size=3, activation='relu')(gender_sec4)
  #gender_sec5 = Conv2D(filters=128, kernel_size=3, activation='relu')(gender_sec5)
  gender_sec5 = BatchNormalization()(gender_sec5)

  gender_sec6 = MaxPooling2D(pool_size = 2)(gender_sec5)
  #gender_sec6 = Concatenate(axis= 1)([gender_sec6,mix_sec6])

  gender_sec7 = Conv2D(filters=64, kernel_size=3, activation='relu')(gender_sec6)
  #gender_sec7 = Conv2D(filters=64, kernel_size=3, activation='relu')(gender_sec7)
  gender_sec7 = BatchNormalization()(gender_sec7)

  gender_sec8 = MaxPooling2D(pool_size=2)(gender_sec7)
  #gender_sec8 = Concatenate(axis= 1)([gender_sec8,mix_sec8])

  gender_sec9 = Conv2D(filters=32, kernel_size=3, activation='relu')(gender_sec8)
  gender_sec9 = Conv2D(filters=32, kernel_size=3, activation='relu')(gender_sec9)
  gender_sec9 = BatchNormalization()(gender_sec9)

  gender_sec10 = Conv2D(filters=32, kernel_size=3, activation='relu')(gender_sec9)
  gender_sec10 = Conv2D(filters=32, kernel_size=3, activation='relu')(gender_sec10)
  gender_sec10 = BatchNormalization()(gender_sec10)

  gender_sec11 = Flatten(name = 'gender_flatten')(gender_sec10)

  gender_sec12 = Dense(1024,activation="relu")(gender_sec11)
  gender_sec12 = Dropout(0.5)(gender_sec12)
  gender_sec12 = Dense(512,activation="relu")(gender_sec12)
  gender_sec12 = Dense(256,activation="relu")(gender_sec12)
  gender_sec12 = Dropout(0.5)(gender_sec12)
  gender_sec12 = Dense(128,activation="relu")(gender_sec12)
  gender_sec12 = Dense(64,activation="relu")(gender_sec12)
  gender_sec12 = Dropout(0.5)(gender_sec12)
  gender_sec12 = Dense(1,activation="sigmoid",name = "Gender_output_layer")(gender_sec12)



  #only age section
  age_sec1 = Conv2D(filters=512, kernel_size=3, activation='relu')(input_image)
  #age_sec1 = Conv2D(filters=512, kernel_size=3, activation='relu')(age_sec1)
  age_sec1 = BatchNormalization()(age_sec1)

  age_sec2 = MaxPooling2D(pool_size=2)(age_sec1)
  #age_sec2 = Concatenate(axis= 1)([age_sec2,mix_sec2])

  age_sec3 = Conv2D(filters=256, kernel_size=3, activation='relu')(age_sec2)
  #age_sec3 = Conv2D(filters=256, kernel_size=3, activation='relu')(age_sec3)
  age_sec3 = BatchNormalization()(age_sec3)

  age_sec4 = MaxPooling2D(pool_size=2)(age_sec3)
  #age_sec4 = Concatenate(axis= 1)([age_sec4,mix_sec4])

  age_sec5 = Conv2D(filters=128, kernel_size=3, activation='relu')(age_sec4)
  #age_sec5 = Conv2D(filters=128, kernel_size=3, activation='relu')(age_sec5)
  age_sec5 = BatchNormalization()(age_sec5)

  age_sec6 = MaxPooling2D(pool_size=2)(age_sec5)
  #age_sec6 = Concatenate(axis= 1)([age_sec6,mix_sec6])

  age_sec7 = Conv2D(filters=64, kernel_size=3, activation='relu')(age_sec6)
  #age_sec7 = Conv2D(filters=64, kernel_size=3, activation='relu')(age_sec7)
  age_sec7 = BatchNormalization()(age_sec7)

  age_sec8 = MaxPooling2D(pool_size=2)(age_sec7)
  #age_sec8 = Concatenate(axis= 1)([age_sec8,mix_sec8])

  age_sec9 = Conv2D(filters=32, kernel_size=3, activation='relu')(age_sec8)
  age_sec9 = Conv2D(filters=32, kernel_size=3, activation='relu')(age_sec9)
  age_sec9 = BatchNormalization()(age_sec9)

  age_sec10 = Conv2D(filters=32, kernel_size=3, activation='relu')(age_sec9)
  age_sec10 = Conv2D(filters=32, kernel_size=3, activation='relu')(age_sec10)
  age_sec10 = BatchNormalization()(age_sec10)

  age_sec11 = Flatten(name = 'age_flatten')(age_sec10)

  age_sec12 = Dense(1024,activation="relu")(age_sec11)
  age_sec12 = Dropout(0.5)(age_sec12)
  age_sec12 = Dense(512,activation="relu")(age_sec12)
  age_sec12 = Dense(256,activation="relu")(age_sec12)
  age_sec12 = Dropout(0.5)(age_sec12)
  age_sec12 = Dense(128,activation="relu")(age_sec12)
  age_sec12 = Dense(64,activation="relu")(age_sec12)
  age_sec12 = Dropout(0.5)(age_sec12)
  age_sec12 = Dense(1,activation="linear",name = "Age_output_layer")(age_sec12)




  model = Model(input_image, outputs= {"gender_output" : gender_sec12, "age_output": age_sec12})
  return model





In [20]:
input_shape = (i_shape, i_shape, 3)
optimzer = Adam(learning_rate=0.001)
# n_classes=2

#model = create_model(input_shape)
model = custom_create_model_given(input_shape)
print(model.summary())


losses = {
	"gender_output": "binary_crossentropy",
	"age_output": "MAE",
}

metrics = {
    "gender_output": "accuracy",
	"age_output": "MAE",
}


print("[INFO] compiling model...")
model.compile(loss=losses, optimizer=optimzer)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_8 (Conv2D)              (None, 222, 222, 51  14336       ['input_1[0][0]']                
                                2)                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 222, 222, 51  14336       ['input_1[0][0]']                
                                2)                                                            

In [21]:
#checkpoint = tensorflow.keras.callbacks.ModelCheckpoint('EfficientNetV2B0.h5',  save_best_only=True,monitor='val_loss',mode='min')

In [22]:
epochs = 1

In [23]:
history = model.fit(train_data_sequence,
                    validation_data = test_data_sequence,
                    epochs = epochs,
                    #batch_size = 16,
                    verbose=1)

 2/34 [>.............................] - ETA: 44:55 - loss: 15.8392 - Age_output_layer_loss: 15.1452 - Gender_output_layer_loss: 0.6941  

KeyboardInterrupt: ignored